In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:
data = pd.read_csv('Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv', encoding='utf-8')
data.head()

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,389,113095465,48,24,9668,10012,403,0,201.416667,203.548293,...,32,203985.500,5.758373e+05,1629110,379,13800000.0,4.277541e+06,16500000,6737603,BENIGN
1,389,113473706,68,40,11364,12718,403,0,167.117647,171.919413,...,32,178326.875,5.034269e+05,1424245,325,13800000.0,4.229413e+06,16500000,6945512,BENIGN
2,0,119945515,150,0,0,0,0,0,0.000000,0.000000,...,0,6909777.333,1.170000e+07,20400000,6,24400000.0,2.430000e+07,60100000,5702188,BENIGN
3,443,60261928,9,7,2330,4221,1093,0,258.888889,409.702162,...,20,0.000,0.000000e+00,0,0,0.0,0.000000e+00,0,0,BENIGN
4,53,269,2,2,102,322,51,51,51.000000,0.000000,...,32,0.000,0.000000e+00,0,0,0.0,0.000000e+00,0,0,BENIGN


In [3]:
data.shape[0]

170366

In [4]:
colnamelist = []
for colname in data.columns:
    if colname[0] == ' ':
        colnamelist.append(colname[1:])
    else:
        colnamelist.append(colname)
data.columns = colnamelist
data['Label'].unique()

array(['BENIGN', 'Web Attack � Brute Force', 'Web Attack � XSS',
       'Web Attack � Sql Injection'], dtype=object)

In [5]:
y = data['Label']
ylabel = y.unique()
ylabel
ylabel_corrected = ['BENIGN', 'Web Attack Brute Force', 'Web Attack XSS',
       'Web Attack Sql Injection']
for i in range(4):
    y[y == ylabel[i]] = i
data['Label'] = y
data['Label'].unique()

/Users/pqros/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


array([0, 1, 2, 3], dtype=object)

In [6]:
#1. Supprimer toutes les lignes contenant au moins une valeur Inf ou NaN. 
data.replace(np.inf, np.nan, inplace=True)
data.replace(-np.inf, np.nan, inplace=True)
data = data.dropna(axis = 0, how = 'any')
data.shape[0]


170346

In [7]:
Z = data.as_matrix().astype(np.float)
np.isnan(Z).any(), np.isinf(Z).any()

(False, True)

In [8]:
mask = []
for i in range(Z.shape[0]):
    if np.isinf(Z[i]).any():
        mask.append(i)
Z = np.delete(Z, mask, axis = 0)
Z.shape[0], np.isinf(Z).any()

(170231, False)

In [9]:
#Construct X and y
X = Z[:, :-1]
y = Z[:, -1]

In [10]:
#2. Supprimer toutes les colonnes avec des valeurs non numériques sauf le label (nom de l'attaque, ou "benign") afin d'obtenir un format du type Z = [ [log, label] ] , où log n'a que des valeurs numériques

#There isn't any non-numeric vaviables in the table except 'Label'

In [11]:
#3. Utiliser la fonction train_test_split de scikit-learn en prenant 70% des flux pour train et 30% pour test. Au résultat, vous devez avoir 4 conteneurs : X_train, Y_train, X_test, Y_test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_train.shape, X_test.shape

((119161, 78), (51070, 78))

In [12]:
#Logistic Regression
lr = LogisticRegression().fit(X_train, y_train)

In [13]:
y_train_ = lr.predict(X_train)
y_test_ = lr.predict(X_test)

In [14]:
from sklearn.metrics import accuracy_score

In [15]:
accuracy_train = accuracy_score(y_train, y_train_)
accuracy_test = accuracy_score(y_test, y_test_)
accuracy_train, accuracy_test

(0.9930933778669195, 0.9932445662815743)

In [16]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier

In [17]:
dt = DecisionTreeClassifier().fit(X_train, y_train)

In [18]:
y_train_ = dt.predict(X_train)
y_test_ = dt.predict(X_test)
accuracy_train = accuracy_score(y_train, y_train_)
accuracy_test = accuracy_score(y_test, y_test_)
accuracy_train, accuracy_test

(1.0, 0.9947327197963579)

In [19]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
rf = DecisionTreeClassifier().fit(X_train, y_train)

In [20]:
y_train_ = rf.predict(X_train)
y_test_ = rf.predict(X_test)
accuracy_train = accuracy_score(y_train, y_train_)
accuracy_test = accuracy_score(y_test, y_test_)
accuracy_train, accuracy_test

(1.0, 0.9946935578617584)

In [21]:
#Support Vector Machine
from sklearn.svm import LinearSVC
SVM = LinearSVC().fit(X_train, y_train)

In [22]:
y_train_ = SVM.predict(X_train)
y_test_ = SVM.predict(X_test)
accuracy_train = accuracy_score(y_train, y_train_)
accuracy_test = accuracy_score(y_test, y_test_)
accuracy_train, accuracy_test

(0.986018915584797, 0.9858821225768553)

## Remarque
Pourquoi le timestamp risque de provoquer une situation de sur-apprentissage dans notre situation ?
Comment peut-on utiliser les variables catégorielles dans nos apprentissages ?

Observez-vous des différences entre les valeurs de métrique sur le set train, et sur le set test ? Une interprétation ?

1. Les données ne sont pas en équilibre : il y a très peu d'échantillons qui sont reconnues comme attaques...

2. On utilise la fonction $get_dummy$ pour les mettre en quelques colomnes de $0 - 1$ variables

3. Les précisions sont trop proche de 1. On peut en déduire peu.